. Fusion et Traitement des Données Démographiques et Géographiques pour Analyse

In [1]:
import pandas as pd

# Définir la fonction pour traiter les données
def store_data(demographic_data, geographic_data):
    # Fusionner les données sur la colonne 'Location'
    merged_data = pd.merge(demographic_data, geographic_data, on='Location')
    # Afficher les données fusionnées
    return merged_data

# Lire les fichiers CSV
demographic_data = pd.read_csv('demographic_data.csv')
geographic_data = pd.read_csv('geographic_data.csv')

# Appeler la fonction avec les deux DataFrames
store_data(demographic_data, geographic_data)


,Location,Population,Density (people/km²),Urbanization (%),Average Income (USD),Education Level (% with Bachelor's or higher),Altitude (m),Proximity to Industry (km),Latitude,Longitude
0,Los Angeles,10039107,3276,89,60000,32,89,5.0,34.052200,-118.243700
1,Paris,2140526,21383,98,45000,29,35,3.0,48.856600,2.352200
2,Tokyo,13929286,6169,100,55000,37,40,2.0,35.689500,139.691700
3,Antananarivo,1391433,3097,69,1000,10,1276,0.5,-18.879200,47.507900
4,Nairobi,4397073,6000,61,1500,12,1795,1.0,-1.286389,36.817223
5,Lima,9674755,3220,81,4500,15,1540,1.5,-12.046400,-77.042800


Automatisation de la Récupération des Données de Pollution Atmosphérique à Partir de Coordonnées Géographiques

In [2]:
import requests
import json
import pandas as pd
import os

# Lire les fichiers CSV
demographic_data = pd.read_csv('demographic_data.csv')
geographic_data = pd.read_csv('geographic_data.csv')

# ajout manuel des colonnes Latitude et Longitude dans geographic_data.csv
# (Idéalement, ces données devraient être obtenues d'une source fiable ou ajoutées manuellement)

# Fusionner les deux jeux de données sur la colonne 'Location' pour obtenir toutes les informations en un seul DataFrame
merged_data = pd.merge(demographic_data, geographic_data, on='Location')

# Créer un dictionnaire de coordonnées à partir des données fusionnées
coordinates = {}
for index, row in merged_data.iterrows():
    city = row['Location']
    lat = row['Latitude']
    lon = row['Longitude']
    coordinates[city] = (lat, lon)

# Récupérer la clé API depuis les variables d'environnement
API_KEY = os.getenv('OPENWEATHER_API_KEY')
BASE_URL = 'http://api.openweathermap.org/data/2.5/air_pollution'

def get_air_pollution_data(lat, lon):
    url = f"{BASE_URL}?lat={lat}&lon={lon}&appid={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

# Récupération des données de pollution pour chaque ville
for city, (lat, lon) in coordinates.items():
    print(f"Récupération des données pour {city}...")
    data = get_air_pollution_data(lat, lon)
    print(json.dumps(data, indent=2))


Récupération des données pour Los Angeles...
{
  "coord": {
    "lon": -118.2437,
    "lat": 34.0522
  },
  "list": [
    {
      "main": {
        "aqi": 3
      },
      "components": {
        "co": 534.06,
        "no": 3.8,
        "no2": 87.74,
        "o3": 0.6,
        "so2": 17.17,
        "pm2_5": 24.7,
        "pm10": 37.7,
        "nh3": 3.01
      },
      "dt": 1723295241
    }
  ]
}
Récupération des données pour Paris...
{
  "coord": {
    "lon": 2.347,
    "lat": 48.859
  },
  "list": [
    {
      "main": {
        "aqi": 3
      },
      "components": {
        "co": 230.31,
        "no": 0.41,
        "no2": 2.36,
        "o3": 107.29,
        "so2": 2.27,
        "pm2_5": 7.52,
        "pm10": 13,
        "nh3": 1.66
      },
      "dt": 1723295275
    }
  ]
}
Récupération des données pour Tokyo...
{
  "coord": {
    "lon": 139.6917,
    "lat": 35.6895
  },
  "list": [
    {
      "main": {
        "aqi": 2
      },
      "components": {
        "co": 393.87,
      

Enrichissement des Données Géographiques avec les Indices AQI et Composants de Pollution Atmosphérique

In [3]:
import requests
import json
import pandas as pd
from datetime import datetime

# Supposons que 'Latitude' et 'Longitude' sont les noms corrects des colonnes
def get_pollution_aqi_data(row, API_KEY):
    lat = row['Latitude']
    lon = row['Longitude']
    url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_KEY}'
    response = requests.get(url)
    data = response.json()
    return data['list'][0]['main']['aqi']

def get_pollution_components_data(row, API_KEY):
    lat = row['Latitude']
    lon = row['Longitude']
    url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_KEY}'
    response = requests.get(url)
    data = response.json()
    return data['list'][0]['components']

# Ajout des données de pollution de l'air dans le DataFrame
merged_data['AQI Pollution'] = merged_data.apply(lambda row: get_pollution_aqi_data(row, API_KEY), axis=1)

# Récupérer les composants de la pollution
components_df = merged_data.apply(lambda row: pd.Series(get_pollution_components_data(row, API_KEY)), axis=1)

# Joindre les composants au DataFrame principal
merged_data = merged_data.join(components_df)

# Supprimer les colonnes lat/lon car elles ne sont plus nécessaires
merged_data.drop(columns=['Latitude', 'Longitude'], axis=1, inplace=True)

# Ajouter la date courante
merged_data['date'] = datetime.now().date()

# Définir 'Location' comme index
merged_data.set_index('Location', inplace=True)

# Afficher les 5 premières lignes du DataFrame
print(merged_data.head(5))


              Population  Density (people/km²)  Urbanization (%)  \
Location                                                           
Los Angeles     10039107                  3276                89   
Paris            2140526                 21383                98   
Tokyo           13929286                  6169               100   
Antananarivo     1391433                  3097                69   
Nairobi          4397073                  6000                61   

              Average Income (USD)  \
Location                             
Los Angeles                  60000   
Paris                        45000   
Tokyo                        55000   
Antananarivo                  1000   
Nairobi                       1500   

              Education Level (% with Bachelor's or higher)  Altitude (m)  \
Location                                                                    
Los Angeles                                              32            89   
Paris                    

## Chargement des Données dans un Fichier

In [4]:
# Enregistrement du rapport
merged_data.to_csv('air_pollution_data.csv')